In [ ]:
from pathlib import Path

PROJ_PATH = str(Path().cwd())

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
# df_4 = pd.read_csv(f"{PROJ_PATH}/Data/Alberta_Wildfire/wildfire-2006-2021.csv")

DF_ORIGINAL = pd.read_csv(
    f"{PROJ_PATH}/Data/Alberta_Wildfire/fp-historical-wildfire-data-2006-2023.csv"
)

In [ ]:
print(DF_ORIGINAL.columns)

In [ ]:
###############
# {Name} {Number} -> 해당 컬럼의 Nan 갯수
##############
identifier = ["fire_year", "fire_number"]
core_info = [
    "current_size",  # target size
    "fire_location_latitude",
    "fire_location_longitude",
    "reported_date",  # time basis
    "bh_fs_date",  # target time
    # "uc_fs_date",
    # "ex_fs_date",
]
# ex_fs_date 25
# bh - being held -
# uc - under control time
# ex - extinguish time,즉, 완전 진화. 이 시점을 쓰는게 베스트이지만, 문제는 데이터 등록이 덜 된 최신 25개 데이터에 대해서는 해당값 비어있음.
# 추가 확인 결과, ex, uc 둘 다 후속 조사의 문제때문인지, 누락, 지연인지,
#  시간 간격이 1년 이상 되는 경우도 발생. -> 직접 활용엔 좀 문제가 있을듯.
# bh 시간 갭의 경우 최대 3개월정도로, reasonable.


date_related = [
    "dispatch_date",
    "start_for_fire_date",
    "assessment_datetime",
]
# nan 값이 50 개 미만이면서 연관성이 있어보이는 정보들

# dispatch_date 17
# start_for_fire_date 20
manynan_date_related = [
    "fire_start_date",
    "discovered_date",
    "ia_arrival_at_fire_date",
    "fire_fighting_start_date",
]
# fire_start_date 678
# discovered_date 4941
# ia_arrival_at_fire_date 7319
# fire_fighting_start_date 7188

input_info = [
    "general_cause_desc",
    "fire_origin",
    "det_agent_type",
    "det_agent",
    "dispatched_resource",
    "assessment_resource",
    "assessment_hectares",
    "initial_action_by",
]
# nan 값이 50 개 미만이면서 연관성이 있어보이는 정보들
# fire_origin 16
# dispatched_resource 17
# initial_action_by 29
important_manynan_input_info = [
    "fire_spread_rate",
    "fire_type",
    "fire_position_on_slope",
    "weather_conditions_over_fire",
    "temperature",
    "relative_humidity",
    "wind_direction",
    "wind_speed",
    "fuel_type",  # Nan can be changed to "others"
    "bucketing_on_fire",  # bool or nan(just make it as False)
    "first_bucket_drop_date",  # must be nan if bucketing_on_fire is False or nan
    "distance_from_water_source",  # must be nan if bucketing_on_fire is False or nan
]
# nan 값이 1,000개 이상이지만, 화재 규모, 지속과 연관이 클 것으로 예상되는 정보들.
# 이 컬럼의 nan 값을 어떻게 처리할지가 의문

# fire_spread_rate 2752
# fire_type 2569
# fire_position_on_slope 2809
# weather_conditions_over_fire 2818
# temperature 2820
# relative_humidity 2822
# wind_direction 2823
# wind_speed 2823
# fuel_type 7353
# bucketing_on_fire 7387
# first_bucket_drop_date 18226
# distance_from_water_source 18228

optional = [
    "discovered_size",
    "industry_identifier_desc",
    "responsible_group_desc",
    "activity_class",
    "true_cause",
    "ia_access",
    "fire_fighting_start_size",
]
# 직접적 연관성이 적어보이거나, nan값이 지나치게 많거나, 특정 column끼리 의존성이 존재하는 경우

no_use = [
    "fire_name",
    "size_class",  # defined by current_size
    "to_fs_date",
    "bh_hectares",
    "uc_hectares",
    "to_hectares",
    "ex_hectares",
]


for i in manynan_date_related:
    print(i, DF_ORIGINAL.loc[:, i].isna().sum())
print("\n")
for i in important_manynan_input_info:
    print(i, DF_ORIGINAL.loc[:, i].isna().sum())
print("\n")
for i in optional:
    print(i, DF_ORIGINAL.loc[:, i].isna().sum())

In [ ]:
columns_to_use = []
columns_to_use += identifier
columns_to_use += core_info
columns_to_use += date_related
columns_to_use += input_info
# 0. Add columns to use
columns_to_use += ["bucketing_on_fire"]
# manynan_date_related
# important_manynan_input_info

df = DF_ORIGINAL.loc[:, columns_to_use]

# 1. Text normalization
for i in [
    "fire_number",
    "general_cause_desc",
    "fire_origin",
    "det_agent_type",
    "det_agent",
    "dispatched_resource",
    "assessment_resource",
    "initial_action_by",
    "bucketing_on_fire",
]:  # list of text columns
    df[i] = df[i].str.lower()

# 2. Preprocess for Many_Nan columns
# Example
df["bucketing_on_fire"].fillna("n", inplace=True)
df["bucketing_on_fire"].replace({"y": True, "n": False}, inplace=True)
df["bucketing_on_fire"] = df["bucketing_on_fire"].astype(bool)


# 3. remove rows which contain Nan
# If you want to keep Nan, convert Nan to real value at stage 2
print("Number of rows to remove due to NaN: ", df.isna().any(axis=1).sum())
df = df.loc[~df.isna().any(axis=1)]


# 4. calculate gap between dates in minutes unit
# Example
timeformat = "%Y-%m-%d %H:%M:%S"

df["bh_gap"] = (
    pd.to_datetime(df["bh_fs_date"], format=timeformat)
    - pd.to_datetime(df["reported_date"], format=timeformat)
).dt.total_seconds() / 60.0

df["dispatch_gap"] = (
    pd.to_datetime(df["dispatch_date"], format=timeformat)
    - pd.to_datetime(df["reported_date"], format=timeformat)
).dt.total_seconds() / 60.0
df["start_for_fire_gap"] = (
    pd.to_datetime(df["start_for_fire_date"], format=timeformat)
    - pd.to_datetime(df["reported_date"], format=timeformat)
).dt.total_seconds() / 60.0
df["assessment_gap"] = (
    pd.to_datetime(df["assessment_datetime"], format=timeformat)
    - pd.to_datetime(df["reported_date"], format=timeformat)
).dt.total_seconds() / 60.0

print(df["bh_gap"].max() / (60 * 24), "days")